In [103]:
from uncertain_dp2 import *

In [104]:
from pot_correction import *

In [105]:
import random
#random.seed(1)

In [106]:
DNA_LENGTH = 30           
READ_lENGTH = 20
READ_COUNT = 10

In [107]:
dna = generate_dna(DNA_LENGTH)
reads = [generate_read(dna, READ_lENGTH, lambda base: gauss_unsharp_uncertainty_generator(base, 0.47)) for _ in range(READ_COUNT)]
# 0.47

In [108]:
most_likely_restorer_error_rate(reads)

[0.85, 0.75, 0.65, 0.85, 0.7, 0.95, 0.75, 0.9, 0.9, 0.8]


0.18999999999999984

In [109]:
read_consent(reads, as_read=True)

          <AATACTTGTCGCGCCGGCCA>

In [110]:
r1 = reads[2]
r1

        <TCGAGACTTGGGGCTCAAGC>

In [111]:
edit_distance(make_regular(*dna), make_regular(*r1.predicted_text))

7

In [112]:
dna_distance_error_rate(dna, reads)

[0.15, 0.25, 0.35, 0.15, 0.3, 0.05, 0.25, 0.1, 0.1, 0.2]


0.19

In [113]:
check_dna_distance_error_rate_suitable(dna, reads)

[0.85, 0.75, 0.65, 0.85, 0.7, 0.95, 0.75, 0.9, 0.9, 0.8]
[0.15, 0.25, 0.35, 0.15, 0.3, 0.05, 0.25, 0.1, 0.1, 0.2]


In [114]:
reads

[          <AATACTTGACGCCCCGGCTA>,
      <ATATAAATACTTTCCGCGCC>,
         <TCGAGACTTGGGGCTCAAGC>,
          <CAAAACTTGATGCGCCGGCC>,
           <CACAGTTGTCGCCCCGGCCG>,
     <TGTAGCAATACTTGTCGCGC>,
          <CGATACGTAGTGCGCCGGCA>,
 <GCTGTGTACCGATACTTGGC>,
           <ACTACTTGGTGCGCCGGCCA>,
           <AACACTTGGGGCCCCCGCCA>]

In [115]:
corrected_reads = correct_reads_with_consens(reads,probabilistic=True)
corrected_reads

[          <AATACTTGACGCCCCGGCTA>,
      <ATATAAATACTTTCCGCGCC>,
         <TCGAGACTTGGGGCTCAAGC>,
          <CAAAACTTGATGCGCCGGCC>,
           <CACAGTTGTCGCCCCGGCCG>,
     <TGTAGCAATACTTGTCGCGC>,
          <CGATACGTAGTGCGCCGGCA>,
 <GCTGTGTACCGATACTTGGC>,
           <ACTACTTGGTGCGCCGGCCA>,
           <AACACTTGGGGCCCCCGCCA>]

In [116]:
DNA_LENGTH = 50           
READ_lENGTH = 35 
READ_COUNT = 10       

In [117]:
most_likely_restorer_error_rate(corrected_reads)

[0.85, 0.75, 0.65, 0.85, 0.7, 0.95, 0.75, 0.9, 0.9, 0.8]


0.18999999999999984

In [118]:
dna_distance_error_rate(dna, reads)

[0.15, 0.25, 0.35, 0.15, 0.3, 0.05, 0.25, 0.1, 0.1, 0.2]


0.19

In [119]:
dna_distance_error_rate(dna, corrected_reads)

[0.15, 0.25, 0.3181818181818182, 0.14285714285714285, 0.3, 0.038461538461538464, 0.25, 0.1, 0.1, 0.2]


0.18181818181818182